As already said in the previous [notebook](15.NYC%20Weather%20Categorical%20Dataset%20Feature%20Engineering.ipynb), many of the weather stations did not reported informations for every days. Otherwise, the number of lines in the *weather* dataset would have been 15'106 (\<number of stations\> X \<number of days\>).


Furtermore, I've identified some case where, even if the weather station reported values for a particular day, some of the me4trics where empty.

What wouold be nice is to have a complete dataset, replacing missing line and metrics with *coherent* remplacement values.


What's the strategy I choose to fill missing metrics ? Extension and extrapolation !!

- **Extension**: will be the process of creating an extended dataset with the 3'619 mising lines counted above, setting feature values to NaN

- **Extrapolation**: will be the process of *extrapolating* the NaN values using non NaN values for the same day.

More details on how to do that trick in the following cells of this Notebook.

Let's go :-)


In [1]:
# Load my_utils.ipynb in Notebook
from ipynb.fs.full.my_utils import *

Opening connection to database
Add pythagore() function to SQLite engine
Fraction of the dataset used to train models: 10.00%
my_utils library loaded :-)


In [2]:
# Load current weather_num table in Dataframe
df=load_sql('weather_num', verbose=False)


In [3]:
# Get number of lines from dataset
number_of_lines=len(df.index)

# Calculate number of lines if all the STATION had reported values
expected_number_of_lines=len(get_stations().index) * len(get_days()) # get_days() comes from my_utils library

# Print the difference
print("Number of missing lines in the weather dataset: ", expected_number_of_lines - number_of_lines)


Number of missing lines in the weather dataset:  3619


In [4]:
# Display the NaN values per feature
print("Number of NaN values per feature in dataset:")
df.isna().sum().sort_values(ascending=False)

Number of NaN values per feature in dataset:


TSTD       9077
TAVG       9077
SNWD       8420
SNOW       4536
PRCP        200
AWND          0
DATE          0
STATION       0
dtype: int64

In [5]:
# Verify SQL tablename is defined in my_utils library
print("Table name used to save the improved dataset:", WEATHER_NUM_TABLENAME)

Table name used to save the improved dataset: weather_num_improved


# Extension: Build an extended dataset

The goal here is to extend the *weather_num* dataset with the 3'619 missing line, filling new line features with np.nan values.
The result will be a dataset of 83 stations x 182 days = 15106 lines.

To do so, I will first create an empty DataFrame with the same columns as *weather_num*.

Then, I will use the *get_days()* function from [my_utils](my_utils.ipynb) library to create a loop on every days, and for each of them i will :

1. Select all the lines for that day from *weather_num* table of my database

2. Append the result to my empty DataFrame

3. Select *STATION* from *stations* table which **are not** in the lines selected in point (1), appended with the current day

4. Append this second query result to my empty DataFrame.

At the end of the loop, I should have a 15'106 lines dataset, one line for each *DATE* and *STATION* :-)

Here are the SQL described above for day = '2016-01-01':

    (1) SELECT * FROM weather_num WHERE DATE='2016-01-01'

    (2) SELECT STATION, '2016-01-01' AS DATE FROM stations WHERE STATION NOT IN ('US1NYWC0003','US1NJBG0023', [...],'USW00054787')

In [6]:
# Create an empty global dataset
stations_and_day_df=pd.DataFrame(columns=df.columns)

# Loop for each days from '2016-01-01' to '2016-01-01'
for date in get_days():
    
    # Select lines for current date loop
    query=f"SELECT * FROM weather_num WHERE DATE='{date}'"
    
    # Store result in temp_df
    temp_df=load_sql(query=query, verbose=False)
    
    # Append temp_df to global dataset
    stations_and_day_df=stations_and_day_df.append(temp_df, ignore_index=True, sort=False)

    # Select STATION that are not in the temp_df dataset, appending 'date' to result
    query="SELECT STATION, '{}' AS DATE FROM stations WHERE STATION NOT IN ('{}')".format(date,"','".join(temp_df['STATION']))

    # Run query and append result to global dataset
    stations_and_day_df=stations_and_day_df.append(load_sql(query=query, verbose=False), ignore_index=True, sort=False)
    
# Copy result in a new dataset that will be used in the cells below: weather_df_extended
weather_df_extended=stations_and_day_df.sort_values(by=['DATE', 'STATION'])


Let's check the number of lines in our dataset, and the number of NaN value per feature.

In [7]:
# Display the number of lines, should be 15'106
print("Number of lines in extended dataset:", len(weather_df_extended.index))

# Display the NaN values per feature
print("Number of NaN values per feature in dataset:")
print(weather_df_extended.isna().sum().sort_values(ascending=False))


Number of lines in extended dataset: 15106
Number of NaN values per feature in dataset:
TSTD       12696
TAVG       12696
SNWD       12039
SNOW        8155
PRCP        3819
AWND        3619
DATE           0
STATION        0
dtype: int64


Well done, I have now a dataset with a line for each *DATE* and *STATION*.

Let's go on with the next part, extrapolation.

# Extrapolation: Fill missing values with extrapolated one

What is *extrapolation* ?

As we are talking here about numerical values that are *continuous*, I've decided to fill in the NaN cells with the average values of the maximum three nearest weather stations of the concerned cells.

To do so, I've written a function, *get_nan_replacement_value()*, which is detailed in the next cell.

This function simply returns the replacement value for a cell using three parameters:
- station: The weather station we are looking for replacement value.
- date: the date of the replacement value.
- feature: The name of the feature we would like to replace.


In [8]:
def get_nan_replacement_value(station, date, feature) -> float:
    """
    Returns a replacement value for the 'feature' received as parameter using the average
    of the maximum three nearest stations, for the 'station' and 'date' received as parameter
    
    Here is the algorythm choosen:
    
    - Performs a SQL SELECT in weather_num where feature is not null.
    - In the previous SELECT, used the pythagore() method two identify the nearest three stations
    - Form the previous result, performs a SELECT AVG(feature). This will calculate the average of 
      the maximum three values from the nearest weather station.
    - Use Dataframe.at() method to retrieve the average value we are looking for, and return it.
    
    Parameters are:
    - station: The weather station we are looking for replacement value.
    - date: the date of the replacement value.
    - feature: The name of the feature we would like to replace.
    
    Returns:
    --------
    float
    
    """

    # First get location of 'station' passed as parameter
    station_df=get_stations(station_list=[station])[['LATITUDE', 'LONGITUDE']]
    station_latitude=float(station_df.LATITUDE.values[0])
    station_longitude=float(station_df.LONGITUDE.values[0])
    
    # Build first SQL queries (the one who gets the 3 nearest weather station with their 'feature' values)
    query='SELECT '
    query+=f"W.STATION, S.LATITUDE, S.LONGITUDE, W.DATE, W.{feature}, '{station_latitude}', '{station_longitude}' "
    query+=f"FROM weather_num AS W "
    query+=f"INNER JOIN stations AS S ON S.STATION=W.STATION "
    query+=f"WHERE DATE='{date}' AND {feature} IS NOT NULL "
    query+=f"ORDER BY pythagore({station_longitude}, {station_latitude}, S.LATITUDE, S.LONGITUDE) ASC LIMIT 3"
    
    # Create second SQL query, based on the previous query, to get average of the feature
    query=f"SELECT avg({feature}) AS {feature} FROM ({query})"
    
    # Get the value we are looking for
    df=load_sql(query=query, verbose=False)
    
   
    return df[feature].values[0]

I'll then loop on each numerical features and apply the *get_nan_replacement_value()* to all cells of the feature that are empty.

In [9]:
# Crete a copy of our extended dataset
weather_num_extrapolated=weather_df_extended.copy()

# Some logging informations
print("== Extrapolation process starting ==")

# Loop on each numerical features
for feature in weather_df_extended.columns[2:]:
    
    # Disaply information on current feature processed
    print("Extrapolating value for feature", feature)
    
    # Display number of NaN values before extrapolation
    print("  Number of NaN values in dataset:",weather_num_extrapolated[feature].isna().sum())
    
    # Build a filter on current feature NaN values
    filter_nan=weather_num_extrapolated[feature].isna()

    # Apply get_nan_replacement_value() on filtered feature
    weather_num_extrapolated[feature]=weather_num_extrapolated.apply(lambda x: get_nan_replacement_value(x['STATION'], x['DATE'], feature) if(pd.notnull(x[feature]) == False) else x[feature], axis=1)

    # Display logging informations
    print("  Processing done")
    
    # Display number of NaN values after extrapolation
    print("  Number of NaN values after extrapolation:",weather_num_extrapolated[feature].isna().sum())

# Display logging informations
print("== Extrapolation process terminated ==")

== Extrapolation process starting ==
Extrapolating value for feature AWND
  Number of NaN values in dataset: 3619
  Processing done
  Number of NaN values after extrapolation: 0
Extrapolating value for feature PRCP
  Number of NaN values in dataset: 3819
  Processing done
  Number of NaN values after extrapolation: 0
Extrapolating value for feature SNOW
  Number of NaN values in dataset: 8155
  Processing done
  Number of NaN values after extrapolation: 0
Extrapolating value for feature SNWD
  Number of NaN values in dataset: 12039
  Processing done
  Number of NaN values after extrapolation: 0
Extrapolating value for feature TAVG
  Number of NaN values in dataset: 12696
  Processing done
  Number of NaN values after extrapolation: 0
Extrapolating value for feature TSTD
  Number of NaN values in dataset: 12696
  Processing done
  Number of NaN values after extrapolation: 0
== Extrapolation process terminated ==


Ok, let's display the first 3 lines of the *weather_num* dataset **before** and **after** the extrapolation, as well as the number of lines and the number of NaN values in the extrapolated dataset.

In [10]:
print("Weather_num dataset BEFORE extrapolation:")
weather_df_extended.head(3)

Weather_num dataset BEFORE extrapolation:


,STATION,DATE,AWND,PRCP,SNOW,SNWD,TAVG,TSTD
77,US1CTFR0022,2016-01-01,NaN,NaN,NaN,NaN,NaN,NaN
79,US1CTFR0039,2016-01-01,NaN,NaN,NaN,NaN,NaN,NaN
38,US1NJBG0002,2016-01-01,0.0,0.3,NaN,NaN,NaN,NaN


In [11]:
print("Weather_num dataset AFTER extrapolation:")
weather_num_extrapolated.head(3)

Weather_num dataset AFTER extrapolation:


,STATION,DATE,AWND,PRCP,SNOW,SNWD,TAVG,TSTD
77,US1CTFR0022,2016-01-01,1.3,0.0,0.0,0.0,4.833333,2.233333
79,US1CTFR0039,2016-01-01,1.3,0.0,0.0,0.0,4.833333,2.233333
38,US1NJBG0002,2016-01-01,0.0,0.3,0.0,0.0,4.833333,2.233333


In [12]:
# Display the number of lines, should be 15'106
print("Number of lines in extended dataset (must be equal to 15'106):", len(weather_num_extrapolated.index))

# Display the NaN values per feature
print("Number of NaN values per feature in dataset (must be all equal to 0):")
print(weather_num_extrapolated.isna().sum().sort_values(ascending=False))


Number of lines in extended dataset (must be equal to 15'106): 15106
Number of NaN values per feature in dataset (must be all equal to 0):
TSTD       0
TAVG       0
SNWD       0
SNOW       0
PRCP       0
AWND       0
DATE       0
STATION    0
dtype: int64


Nice, NaN values have been replaced by extrapolated ones.

And trust me, the *get_nan_replacement_value()* function works the way I wanted to ;-)

# Save extrapolated dataset to SQL database

It's now time to save the result as a new dataset in our SQL database.

In [13]:
# Save extrapolated dataset to SQL database
save_sql(dataset=weather_num_extrapolated, tablename=WEATHER_NUM_TABLENAME)

Saving OK


True

Time to continue with the next notebook: [The global Dataset - Merging all the datasets into a big one](17.The%20global%20Dataset%20-%20Merging%20all%20the%20datasets%20into%20a%20big%20one.ipynb)